# Dokumentation MyCroft AI 

## Verfasser Philipp Umscheid, Rainer Bürg 

## Bedschreibung der Aufgabe: 

Ein eigener Skill: "What's my next appointment?"
Die Mycrof.ai Webseite enthält eine gute Anleitung zur Erstellung eigener Skills: https://mycroft-ai.gitbook.io/docs/skill-development/voice-user-interface-design-guidelines
Machen Sie sich also zunächst mit dem Hintergrund der Skill-Entwicklung vertraut (aka rtfmf). Wenn Sie das Gefühl haben, dass Sie es nicht mehr erwarten können, einen eigenen Skill zu entwickeln (bzw. Sie am Ende des Semsters Ihre Prüfungsleitung bestehen wollen), dann beginnen Sie die Entwicklung des folgenden Skills.


Was soll möglich werden?
Nehmen wir an, dass ein Kunde seine Termine im Calendar-PlugIn des nextcloud-Systems verwaltet (wie zum Beispiel der Autor dieser Zeilen). Er möchte gerne folgende Interaktion realisiert bekommen:

Mensch: "Hey Jarvis, what's my next appointment?"

Jarvis: "Your next appointment is on June 22, 2020 at 4 pm and is entitled Speech Interaction class."

### Umsetzung der Aufgbae 

1. Durchführung des Nextcloud Kalendaraufrufes über die Datei "caldav_nextcloud_starterpy.py"

1.1 Herstellung der Verbindung über die Initilaien des SI-Studierenden

In [1]:
# Verbindungsaufbau zur Nextcloud der Veranstaltung SI20/21
url = "https://" + username + ":" + password + "@next.social-robot.info/nc/remote.php/dav"
# open connection to calendar
client = caldav.DAVClient(url)
principal = client.principal()
calendars = principal.calendars()

NameError: name 'username' is not defined

Nach dem erfolgreichen Verbindungsaufbau und Abfragen von Events wurde der anschließende Code aus der Datei "caldav_nextcloud_starterpy.py" in den späteren MyCroft Skill übertragen. 

2. Einlesen in die MyCroftSkill Architektur 
Die Infos dazu wurden aus den Inhalten der MyCroft AI Dokumentation genommen https://mycroft-ai.gitbook.io/docs/

3. Erstellung eines leeren Skills, welcher mit dem GitHub Account verknüpft ist. 
Erstellung des Skills über das Mycroft Kommando "msk"
(siehe detailierte Beschreibung unter https://mycroft-ai.gitbook.io/docs/skill-development/introduction/your-first-skill)
**Beachten Sie das Problem der Skillerstellung am Ende dieser Dokumentation**

4. Bearbeitung / Erstllung des Skills "nextappointment"
Der Aufruf der Caldav Kalenderdaten wurde in die Datei "\_init\_.py" übertragen. 

Die Datei "\_init\_'.py" stellt das Herzstück des MyCroft Skills dar. Dort werden die verschiedenen Eingaben / "Utterances" des Nutzers auf bestimmte Wortfolgen/ Phrasen überprüft und durch die "Handler"- Funktionen bearbeitet. Am Ende wird dem Nutzer die gewünschte Ausgabe per Dialog zurückgegeben. 



In [ ]:
 @intent_file_handler('nextapp.intent')
    def handle_nextapp(self, message):
        try:
            #Überprüfung des Kalenders ob Events am heutigen Tag eingetragen sind.
            if len(calendars) > 0:
                calendar = calendars[0]
                events = calendar.date_search(today,
                                              datetime.combine(today, time(23, 59, 59, 59)))  # Events am heutigen Tag
                #Falls keine Events vorhanden sind, soll die Ausgabe "Keine Events eingetragen" kommen.
                if len(events) == 0:
                    self.speak_dialog("no_event")
                else:
                    #Aufllistung der heutigen Events:
                    print("Total {num_events} events:".format(num_events=len(events)))
                    listNextAppointments = []

Dort wird der Kalendar am heutigen Tag auf Events überprüft

In [ ]:
                    for event in events:
                        event.load()
                        e = event.instance.vevent
                        #Einlesen der Eventzeit
                        eventTime = e.dtstart.value.strftime("%H:%M")
                        eventVal = "{eventTime} {eventSummary}".format(eventTime=eventTime,
                                                                       eventSummary=e.summary.value)
                        #Überprüfung der aktuellen Zeit.
                        aktuelleZeit = datetime.now().strftime("%H:%M")
                        #Falls der Termin nach der aktuellen Uhrzeit ist wird er der Liste hinzugefügt.
                        if aktuelleZeit < eventTime:
                            listNextAppointments.append((eventVal))
                            print(listNextAppointments)
                        # Falls ein Tagesevent stattfindet soll dies auch dem Nutzer mitgeteilt werden.
                        elif e.dtstart.value.strftime("%H:%M") == "00:00":
                            dailyEventSum = "{eventSummary}".format(eventSummary=e.summary.value)
                            response = {'dailyEventDialog':dailyEventSum}
                            self.speak_dialog('daily_event', data = response)

Es findet eine Unterscheidung zwischen Tagesevents und zeitlich festgelgten Events statt. Dadurch wird einer Liste alle Events weitergeben, welche am heutigen Tag noch stattfinden werden. Diese werden sortiert und der nächste Termin wird nur dem Nutzer ausgegeben. 

In [ ]:
                        else:
                        #Falls kein weiterer Termin heute mehr vorhanden ist, soll demm Nutzer gesagt werden, dass
                        # alle Events schon durch sind.
                            self.speak_dialog('events_done')
                    listNextAppointments.sort()
                    # Rückgabewert an den Nutzer wird das nächste Event sein.
                    nextAppointmentValue = listNextAppointments[0]
                    response ={'nextAppointment':nextAppointmentValue}
                    #Die vordefinierte Antwort + das nächste Event werden von MyCroft an den Nutzer übergeben..
                    self.speak_dialog('nextapp', data=response)

### Funktionsweise der Rückgabe an den Nutzer

Wir haben als Rückgabewert die Uhrzeit + den Eventnamen als String ermittelt und geben diesen String dem Dialog Objekt als Keyvalue zurück  

In [ ]:
response ={'nextAppointment':nextAppointmentValue}
self.speak_dialog('nextapp', data=response)

Dieses "data" Objekt wird dem Dialog "nextapp" übergeben. Dabei ist es wichtig, dass die "Key"-Bezeichnung die gleiche ist, wie in der Dialog-Datei selbst. 

**Inhalt der Datei "nextapp.dialog":** 

This is your next appointment today {nextAppointment}

You have appointments on {nextAppointment} today

The next thing today is {nextAppointment}

**Somit würde ein Dialog zwischen MyCroft und dem Nutzer folgendes ergeben:**  

User: "Hey mycroft, whats my next appointment?"

Mycroft: "This is your next appointment today {nextAppointment} "
    
**Annnahme es gibt ein Event am 05.02.2021 mit dem Titel "Abgabe Si" um "18 Uhr"** 

Mycroft: "This is your next appointment today 18:00 Abgabe Si "



### Bonusaufgabe Abfrage an einem bestimmten Datum

Für die Datumsabfrage wurde ein weiterer Intenthandler dem Skill hinzugefügt, welcher die gleiche Logik für die Rückgabe der Events besitzt. Dabei wurde noch darauf geachtet, dass auch hier wieder zwischen Tagesevents und zeitliche Events unterschieden wird. Nur bei dieser Abfrage werdne alle Events ausgegeben und nicht nur das nächste Event.

Dabaei galt es darauf zu achten, dass der User MyCroft ein Datum vorgibt. Dieses soll im Intent erhalten werden. 


**Inhalt der Datei "events_at.intent":** 

events at {datum}

Das erhaltene Datum wird in Form eines Stringvalues an MyCroft übergeben. Zur bestimmung des Kalendartages / Zeitraumes wird der Wert zu einem Datum formatiert und als Start- und Endzeit festgelgt.

In [ ]:
@intent_file_handler('events_at.intent')
    def handle_events_at(self, message):
        #Abfrage des gesuchten Datums aus der verbalen Nutzer Anfrage / des Nutzerintents
        intentDatum = message.data.get('datum')
        setDatum = datetime.strptime(intentDatum, "%d-%m-%Y")
        endTime = datetime.combine(setDatum, time(23, 59, 59, 59))
        ...
        ...
        events = calendar.date_search(start=setDatum,end=endTime)

Dadurch wir eben der bestimmte Tag nur überprüft und die dortigen Events wie auch bei der Vorherigen Aufgabe gefiltert und dementsprechend wieder als Dialog zurückgeben. Mit der fast gleichen Logik des Codes.  

## Fertiger Skill 

Der erstellte Skill ist momentan unter dem Repro https://github.com/RB072/nextapp-skill.git zu finden.

Der Skill kann einfach im Verzeichnis /mycroft-core mit dem Befehl: "msm install https://github.com/RB072/nextapp-skill.git"
installieren und dann eben auch aus Sicherheitsgrüden die Credentials "username" und "password" für den Verbindungsaufbau ändern. 

### Probleme bei der Bearbeitung

**Caldav Installation:**

Installation des Python Caldav Modul auf dem Raspberry PI. Dies konnte mit dem Kommando "./mycroft-pip install caldav" im bin-Verzeichnis gelöst werden. Wichtig dabei ist das "./" nicht zu vergessen. 

**Skillerstellung:**

Auch die Erstellung der Skills über den Befehl "mycroft-msk create {skillname}" funktionierte nach der Online-Dokumentation nicht ohne Probleme. Unsere Lösung dafür war es auf dem RaspberryPi in das Verzeichnis der Skills "(/mycroft-core/skills)" direkt zu gehen und dort nur den Befehl "msk create {skillname}" einzugeben und auszuführen. 

**Die Datumserkennug:**

Bei der verbalen Eingabe über das Mikrofon an MyCroft konnte er oftmals nicht meine Datumseingabe als "Utterance" verstehen. Wörter / Sätze wurden ohne Problem verstanden nur bei Zahlenfolgen tat sich die Erkennung schwer. Daher wurde die Datumseingabe immer händisch in der Console eingegeben mit z.B. "events at 28-01-2021"

